<a href="https://colab.research.google.com/github/deepavasanthkumar/deepcodesnippets/blob/master/idf_and_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Formula :
idf(t) = log(N/(df + 1))
tf-idf now is a the right measure to evaluate how important a word is to a document in a collection

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 198 kB 64.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=69a8acbdb032a77616acf621b1553c9b8109d90019876ee7883c264838674fc2
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Upload the files


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving inputDoc.txt to inputDoc.txt


 Read the Dataframe
 Assign row_number for document id

 Split with ' '

 


In [ ]:
import math
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("inviz-if-idf").getOrCreate()
  

file =   spark.sparkContext.textFile("inputDoc.txt")

df = spark.read.csv("inputDoc.txt")


w = Window().orderBy(lit('A'))
df = df.withColumn("row_num", row_number().over(w))


df.rdd.collect()
map1=df.rdd.flatMap(lambda x: [((x[1],i),1) for i in x[0].split()])
map1.toDF().show()
reduce=map1.reduceByKey(lambda x,y:x+y)
print('reduce')
reduce.toDF().show()
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
print('tf')
tf.toDF().show()
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()
print('map3')
map3.toDF().show()
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()
print('map4')
map4.toDF().show()
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()
print('reduce2')
reduce2.toDF().show()
lenth = file.count()

idf=reduce2.map(lambda x: (x[0],math.log10(lenth/x[1])))
idf.collect()
idf.toDF().show()
rdd=tf.join(idf)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()
print('rdd')
rdd.toDF().show()
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show()

+--------------+---+
|            _1| _2|
+--------------+---+
|     {1, This}|  1|
|       {1, is}|  1|
|    {1, first}|  1|
|{1, sentence.}|  1|
|     {1, This}|  1|
|       {1, is}|  1|
|        {1, a}|  1|
|     {1, long}|  1|
| {1, sentence}|  1|
|     {2, This}|  1|
|       {2, is}|  1|
|   {2, second}|  1|
| {2, sentence}|  1|
|     {3, This}|  1|
|       {3, is}|  1|
|    {3, third}|  1|
| {3, sentence}|  1|
+--------------+---+

reduce
+--------------+---+
|            _1| _2|
+--------------+---+
|     {1, This}|  2|
|       {1, is}|  2|
|    {1, first}|  1|
|{1, sentence.}|  1|
|        {1, a}|  1|
|     {1, long}|  1|
| {1, sentence}|  1|
|     {2, This}|  1|
|       {2, is}|  1|
|   {2, second}|  1|
| {2, sentence}|  1|
|     {3, This}|  1|
|       {3, is}|  1|
|    {3, third}|  1|
| {3, sentence}|  1|
+--------------+---+

tf
+---------+------+
|       _1|    _2|
+---------+------+
|     This|{1, 2}|
|       is|{1, 2}|
|    first|{1, 1}|
|sentence.|{1, 1}|
|        a|{1, 